# Deploy a PyTorch Model as a Web Service Azure ML

IMPORTANT:
* Please use the **"Python 3.6 - Azure ML" kernel** on the DSVM for this notebook or install appropriate library versions below (to change a kernel go to Kernel in the menu bar and select "Change kernel").
* You will need your `config.json` from your Azure ML Workspace in the **same folder** as this notebook and interactive login will be performed later on so be prepared with your Azure login info.  You may wish to work with these notebooks in an **Incognito or Private browser window** in case you have other Azure accounts.

In [ ]:
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies 

from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
ws = Workspace.from_config(path='config.json')

Provide the experiment suffix used in the training notebook.  Replace `***`.

In [ ]:
my_nickname = ***


## Deploy model as web service
Once you have your trained model, you can deploy the model on Azure. In this tutorial, we will deploy the model as a web service in Azure Container Instances (ACI). For more information on deploying models using Azure ML, refer to [Deploy models with Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

**Create scoring script**

First, we will utilize a pre-made scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions (take a look at `pytorch_score.py`, now):

1. `init()`: In this function, you typically load the model into a global object. This function is executed only once when the Docker container is started.


2. `run(input_data)`:  In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.


Refer to the scoring script `pytorch_score.py` for this tutorial. Our web service will use this file to predict on the new image that is sent as a REST call. When writing your own scoring script, don't forget to test it locally first before you go and deploy the web service.  This will make debugging easier.

**Create environment file**

As part of deploying, we need more than our Python scoring script.  The service is based on a docker image that encapsulates the entire set of requirements.  We will need to create an environment file (`myenv.yml`) that specifies all of the scoring script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image by Azure ML. In this case, we need to specify `azureml-core`, `torch` and `torchvision`.  Let's make sure to be consistent in versions.

Beyond `conda` dependencies, we can also specify `pip` installable dependencies (not shown here).

### Create environment file

In [ ]:
myenv = CondaDependencies.create(pip_packages=['azureml-defaults==1.5.0', 
                                               'torch==1.3.0', 
                                               'torchvision==0.4.1',
                                               'Pillow==6.2.1'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

### Get registered model

Note, you can update the `version` to match the latest one you wish to use.

In [ ]:
model = Model(ws,'behavior-pytorch-'+my_nickname, version=1)#.download(exist_ok=True)
print(model.name, model.id, model.version, sep='\t')

### Set up environment

A good reference notebook from Azure ML team on environments can be found here:  https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/using-environments/using-environments.ipynb.

In [ ]:
myenv = Environment.from_conda_specification(name='myenv',
                                             file_path='myenv.yml')


myenv.environment_variables = {'MODEL_NAME': 'behavior-pytorch-'+my_nickname,
                               'VERSION': 1}
myenv.register(workspace=ws)

### Set up inference config

Create an inference configuration which gives specifies the inferencing environment and scripts.  This will be used in the local deployment as well as the deployment to ACI.

In [ ]:
inference_config = InferenceConfig(entry_script="pytorch_score.py",
                                   environment=myenv)

### Deploy locally to test - may take some time

Always a good idea.

IMPORTANT:  You will need to be able to run `docker` without `sudo` on the machine where you run this notebook.  If you get a "Permission denied" error please follow the guidance (https://docs.docker.com/engine/install/linux-postinstall/#manage-docker-as-a-non-root-user) to create a docker group and add the user, then restart the JupyterHub server (check out the Control Panel).

Building the docker image for the first time may take ~15 minutes depending upon your compute resource.

In [ ]:
# Set up deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=6789)

# Create service
local_service = Model.deploy(ws, 'test', [model], inference_config, deployment_config)
local_service.wait_for_deployment()

In [ ]:
local_service.get_logs()

Send a test image to the local service.

In [ ]:
import numpy as np
from PIL import Image
import json

input_data = Image.open('test_images/Fight_OneManDown321.jpg')
input_data = np.asarray(input_data)
result = local_service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)


input_data = Image.open('test_images/Browse10986.jpg')
input_data = np.asarray(input_data)
result = local_service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)

### Deploy to ACI container
We are ready to deploy. Create a deployment configuration file to specify the number of CPUs and gigabytes of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 3 gigabyte of RAM is usually sufficient for many models. This cell will run for about 5-20 minutes.

Note:  service names must be unique.  Delete service or rename if you wish to deploy another.

In [ ]:

myenv = Environment.from_conda_specification(name='myenv',
                                             file_path='myenv.yml')

# Clear out vars for the ACI version of deployment
myenv.environment_variables = {'MODEL_NAME': '',
                               'VERSION': ''}
myenv.register(workspace=ws)

# Recreate inference config since we changed the environment
inference_config = InferenceConfig(entry_script="pytorch_score.py",
                                   environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=3, 
                                               tags={'data': 'suspicious-behavior',  
                                                     'method':'transfer learning', 
                                                     'framework':'pytorch'},
                                               description='Classify normal/suspicious behavior in PyTorch')

service = Model.deploy(workspace=ws, 
                           name='aci-suspicious-behavior-1', 
                           models=[model],
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.state)

If your deployment fails for any reason and you need to redeploy, make sure to delete the service before you do so: `service.delete()` - see "Clean up" section below.

**Tip**: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:

In [ ]:
service.get_logs()

Get the web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

### Test the web service
Finally, let's test our deployed web service. We will send the data as a JSON string to the web service hosted in ACI and use the SDK's run API to invoke the service. Here we will take an image from our validation data to predict on.

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline
plt.imshow(Image.open('test_images/Fight_OneManDown321.jpg'))

In [ ]:
import numpy as np
input_data = Image.open('test_images/Fight_OneManDown321.jpg')
input_data = np.asarray(input_data)
result = service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)

## Exercise

1. Use the packaging models functionality of Azure ML to package the model as set of prerequisite files to inspect and build custom docker image.
2. Build the image from this notebook or from Terminal.
3. Run the container from this notebook or from Terminal.
4. (BONUS) Test the local container using the `requests` library as shown in the documentation.



See [Generate a Dockerfile and dependencies](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where#generate-a-dockerfile-and-dependencies).

## Clean up
Once you no longer need the web service, you can delete it with a simple API call.

In [ ]:
service.delete()

## Update service

To update the service with the SDK, a good guide can be found in the docs:  https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-azure-container-instance#update-the-web-service.